In [1]:
from MovieRecommend import Movies

In [2]:
mov = Movies()

Midsommar...
importing The Dark Tower...
importing Focus...
importing Point Break...
importing Once Upon a Time in Hollywood...
importing Eddie the Eagle...
importing Stranger Than Fiction...
importing Kung Fu Panda 2...
importing Kung Fu Panda 3...
importing The Prestige...
importing Paul Blart: Mall Cop 2...
importing Good Morning, Vietnam...
importing Game Night...
importing Andhadhun...
importing Howl's Moving Castle...
importing Ad Astra...
importing Walk of Shame...
importing Robin Hood...
importing El Camino: A Breaking Bad Movie...
importing Spirited Away...
importing Gemini Man...
importing Snowpiercer...
importing Geostorm...
importing The Night Before...
importing Frozen...
importing Constantine...
importing Due Date...
importing Klaus...
importing Enter the Dragon...
importing Mr. Right...
importing The Dukes of Hazzard...
importing A Fish Called Wanda...
importing Happy Death Day...
importing The Knight Before Christmas...
importing Chappie...
importing The King's Speech..

In [3]:
mov.print_files()

29  Stephen Schiff (screenplay by), Michael Finch ...   
330                                     Stuart Beattie   
331  Walter Hill (story), Harry Kleiner (screenplay...   
332  Scott Wiper (screenplay), Rob Hedden (screenpl...   
333  Lexi Alexander (story), Dougie Brimson (story)...   

                                                Actors  \
0    Michelle Williams, Christopher Plummer, Mark W...   
1    Dwayne Johnson, Kevin Hart, Jack Black, Karen ...   
2    Keanu Reeves, Riccardo Scamarcio, Ian McShane,...   
3    Jessica Chastain, Idris Elba, Kevin Costner, M...   
4    Gary Oldman, Kristin Scott Thomas, Ben Mendels...   
..                                                 ...   
329  Dylan O'Brien, Charlotte Vega, Christopher Bom...   
330  Tom Cruise, Jamie Foxx, Jada Pinkett Smith, Ma...   
331  Arnold Schwarzenegger, Jim Belushi, Peter Boyl...   
332  Steve Austin, Vinnie Jones, Robert Mammone, To...   
333  Elijah Wood, Charlie Hunnam, Claire Forlani, M...   

             

In [4]:
mov.preview_files()

                            Title  Year  Rated     Released  Runtime  \
0      All the Money in the World  2017      R  25 Dec 2017  132 min   
1  Jumanji: Welcome to the Jungle  2017  PG-13  20 Dec 2017  119 min   
2            John Wick: Chapter 2  2017      R  10 Feb 2017  122 min   
3                    Molly's Game  2017      R  05 Jan 2018  140 min   
4                    Darkest Hour  2017  PG-13  22 Dec 2017  125 min   

                                        Genre        Director  \
0  Biography, Crime, Drama, Mystery, Thriller    Ridley Scott   
1          Action, Adventure, Comedy, Fantasy     Jake Kasdan   
2                     Action, Crime, Thriller  Chad Stahelski   
3                     Biography, Crime, Drama    Aaron Sorkin   
4              Biography, Drama, History, War      Joe Wright   

                                              Writer  \
0  David Scarpa, John Pearson (based on the book ...   
1  Chris McKenna (screenplay by), Erik Sommers (s...   
2  Derek

In [5]:
new_df = mov.get_df()

In [6]:
print(new_df['Writer'])

0      David Scarpa, John Pearson (based on the book ...
1      Chris McKenna (screenplay by), Erik Sommers (s...
2      Derek Kolstad, Derek Kolstad (based on charact...
3      Aaron Sorkin (written for the screen by), Moll...
4                                       Anthony McCarten
                             ...                        
329    Stephen Schiff (screenplay by), Michael Finch ...
330                                       Stuart Beattie
331    Walter Hill (story), Harry Kleiner (screenplay...
332    Scott Wiper (screenplay), Rob Hedden (screenpl...
333    Lexi Alexander (story), Dougie Brimson (story)...
Name: Writer, Length: 334, dtype: object


In [7]:
import re
new_df['Writer'] = new_df['Writer'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", str(x)))
print(new_df['Writer'])

new_df['Writer'] = new_df['Writer'].str.replace(' ','')
new_df['Writer'] = new_df['Writer'].str.replace('N/A','nan')
new_df['Writer'] = new_df['Writer'].apply(lambda x: x.lower())
new_df['Writer'] = new_df['Writer'].apply(lambda x: re.split(',|/',x))
print(new_df['Writer'])

0                            David Scarpa, John Pearson 
1      Chris McKenna , Erik Sommers , Scott Rosenberg...
2                          Derek Kolstad, Derek Kolstad 
3                            Aaron Sorkin , Molly Bloom 
4                                       Anthony McCarten
                             ...                        
329    Stephen Schiff , Michael Finch , Edward Zwick ...
330                                       Stuart Beattie
331    Walter Hill , Harry Kleiner , Walter Hill , Tr...
332    Scott Wiper , Rob Hedden , Rob Hedden , Andy H...
333    Lexi Alexander , Dougie Brimson , Dougie Brims...
Name: Writer, Length: 334, dtype: object
0                             [davidscarpa, johnpearson]
1      [chrismckenna, eriksommers, scottrosenberg, je...
2                           [derekkolstad, derekkolstad]
3                              [aaronsorkin, mollybloom]
4                                      [anthonymccarten]
                             ...               

In [8]:
test = new_df.copy()

form_cols = ['Actors']
print(test['Actors'])

for col in form_cols:
            test[col] = test[col].str.replace(' ','')
            test[col] = test[col].str.replace('N/A','nan')
            test[col] = test[col].apply(lambda x: x.lower())
            test[col] = test[col].apply(lambda x: re.split(',|/',x))

print(test['Actors'])

0      Michelle Williams, Christopher Plummer, Mark W...
1      Dwayne Johnson, Kevin Hart, Jack Black, Karen ...
2      Keanu Reeves, Riccardo Scamarcio, Ian McShane,...
3      Jessica Chastain, Idris Elba, Kevin Costner, M...
4      Gary Oldman, Kristin Scott Thomas, Ben Mendels...
                             ...                        
329    Dylan O'Brien, Charlotte Vega, Christopher Bom...
330    Tom Cruise, Jamie Foxx, Jada Pinkett Smith, Ma...
331    Arnold Schwarzenegger, Jim Belushi, Peter Boyl...
332    Steve Austin, Vinnie Jones, Robert Mammone, To...
333    Elijah Wood, Charlie Hunnam, Claire Forlani, M...
Name: Actors, Length: 334, dtype: object
0      [michellewilliams, christopherplummer, markwah...
1      [dwaynejohnson, kevinhart, jackblack, karengil...
2      [keanureeves, riccardoscamarcio, ianmcshane, r...
3      [jessicachastain, idriselba, kevincostner, mic...
4      [garyoldman, kristinscottthomas, benmendelsohn...
                             ...               

In [9]:
mov.create_features()

In create features
Cleaning writer column
Dataframe assigned
Writer column cleaned
Dataframe assigned
Writer column cleaned and assigned
Dataframe assigned for features
Plot lemma column created
Form columns split
Created new dataframes of split columns
Joining dataframes
Dataframes joined


In [10]:
mov.preview_files()

                            Title  Year  Rated     Released  Runtime  \
0      All the Money in the World  2017      R  25 Dec 2017  132 min   
1  Jumanji: Welcome to the Jungle  2017  PG-13  20 Dec 2017  119 min   
2            John Wick: Chapter 2  2017      R  10 Feb 2017  122 min   
3                    Molly's Game  2017      R  05 Jan 2018  140 min   
4                    Darkest Hour  2017  PG-13  22 Dec 2017  125 min   

                                          Genre         Director Writer  \
0  [biography, crime, drama, mystery, thriller]    [ridleyscott]     []   
1          [action, adventure, comedy, fantasy]     [jakekasdan]     []   
2                     [action, crime, thriller]  [chadstahelski]     []   
3                     [biography, crime, drama]    [aaronsorkin]     []   
4              [biography, drama, history, war]      [joewright]     []   

                                              Actors  \
0  [michellewilliams, christopherplummer, markwah...   
1  [

In [11]:
df = mov.get_df()

In [12]:
mov.print_files()

pter 2  2017      R  10 Feb 2017  122 min   
3                      Molly's Game  2017      R  05 Jan 2018  140 min   
4                      Darkest Hour  2017  PG-13  22 Dec 2017  125 min   
..                              ...   ...    ...          ...      ...   
329               American Assassin  2017      R  15 Sep 2017  111 min   
330                      Collateral  2004      R  06 Aug 2004  120 min   
331                        Red Heat  1988      R  17 Jun 1988  104 min   
332                   The Condemned  2007      R  27 Apr 2007  113 min   
333          Green Street Hooligans  2005      R  09 Sep 2005  109 min   

                                            Genre         Director Writer  \
0    [biography, crime, drama, mystery, thriller]    [ridleyscott]     []   
1            [action, adventure, comedy, fantasy]     [jakekasdan]     []   
2                       [action, crime, thriller]  [chadstahelski]     []   
3                       [biography, crime, drama]    [